# Imports and device

In [1]:
import sys
import os
sys.path.append(os.path.abspath("src"))
import torch
from src.trainer import Trainer
from src.models import NN_CartPole, NN_Humanoid, NN_Hopper

if torch.cuda.is_available():
    device = torch.device("cuda:0")
elif torch.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print("You are using device: %s" % device)

You are using device: mps


# Environment selection

In [2]:
# str_env = "Humanoid-v5"
str_env = "Hopper-v5"
# str_env = "CartPole-v1"

# Configurations and initialisation

In [3]:
updates = {"CartPole-v1": 100, "Humanoid-v5": 1000, "Hopper-v5": 600}
epochs = {"CartPole-v1": 8, "Humanoid-v5": 8, "Hopper-v5": 12} # Hopper-v5: [8, 12, 16]
N = {"CartPole-v1": 4, "Humanoid-v5": 4, "Hopper-v5": 4} # Hopper-v5: [2, 4, 6]
T = {"CartPole-v1": 256, "Humanoid-v5": 256, "Hopper-v5": 1024} # Hopper-v5: [512, 1024, 2048]
batches = {"CartPole-v1": 4, "Humanoid-v5": 4, "Hopper-v5": 4}
value_loss_coef = {"CartPole-v1": 0.5, "Humanoid-v5": 0.5, "Hopper-v5": 1.0}
entropy_bonus_coef = {"CartPole-v1": 1e-3, "Humanoid-v5": 1e-3, "Hopper-v5": 5e-5} # Hopper-v5: [1e-5, 5e-5, 1e-4]
clip_range = {"CartPole-v1": 0.1, "Humanoid-v5": 0.1, "Hopper-v5": 0.1}
learning_rate = {"CartPole-v1": 1e-3, "Humanoid-v5": 1e-3, "Hopper-v5": 1e-4} # Hopper-v5: [1e-5, 1e-4, 1e-3]
learning_rate_decay = {"CartPole-v1": 0.999, "Humanoid-v5": 0.999, "Hopper-v5": 0.999}
reward_scaling = {"CartPole-v1": 1.0, "Humanoid-v5": 0.005, "Hopper-v5": 0.1}
models = {"CartPole-v1": NN_CartPole, "Humanoid-v5": NN_Humanoid, "Hopper-v5": NN_Hopper}

# Configurations
configs = {
    # Number of updates
    'updates': updates[str_env],
    # Number of epochs to train the model with sampled data.
    'epochs': epochs[str_env],
    # Number of worker processes
    'N': N[str_env],
    # Number of steps to run on each process for a single update
    'T': T[str_env],
    # Number of mini batches
    'batches': batches[str_env],
    # Value loss coefficient.
    'value_loss_coef': value_loss_coef[str_env],
    # Entropy bonus coefficient.
    'entropy_bonus_coef': entropy_bonus_coef[str_env],
    # Clip range.
    'clip_range': clip_range[str_env],
    # Learning rate.
    'learning_rate': learning_rate[str_env],
    # Learning rate decay.
    'learning_rate_decay': learning_rate_decay[str_env],
    # Model to use
    'model': models[str_env](),
    # Device to use for training
    'device': device,
    # Environment to use
    'str_env': str_env,
    # Reward scaling
    'reward_scaling': reward_scaling[str_env]
}

# Initialize the trainer
trainer = Trainer(**configs)

# Optional: load a checkpoint pretrained model and log a video

In [4]:
# # Load checkpoint and save video
# checkpoint = torch.load("checkpoint/" + str_env + "_model.pth", weights_only=True, map_location=device)
# trainer.model.load_state_dict(checkpoint)
# if device != torch.device("cuda:0") or str_env == "CartPole-v1":
#     trainer.log_video(str_env)

# Log a video example of the noised agent

In [5]:
if device != torch.device("cuda:0") or str_env == "CartPole-v1":
    trainer.log_video_with_noise(str_env, use_model=False)

Video recorded with duration 300 frames.
MoviePy - Building video results/Hopper-v5_noised.mp4.
MoviePy - Writing video results/Hopper-v5_noised.mp4



MoviePy - Done !
MoviePy - video ready results/Hopper-v5_noised.mp4


# Training with noise

In [ ]:
# Run and monitor the experiment
trainer.run_training_loop_with_noise()

# Stop the workers
trainer.destroy()

 66%|██████▌   | 394/600 [22:06<11:59,  3.50s/it]

# Log a video example of your trained agent and save the model's weights

In [ ]:
if device != torch.device("cuda:0") or str_env == "CartPole-v1":
    trainer.log_video_with_noise(str_env + "_trained", use_model=True)

# Save the model
torch.save(trainer.model.state_dict(), "checkpoint/" + str_env + "_model_with_noise.pth")

# Make the plottings for your single run

In [ ]:
trainer.plot(str_env + "_noised")